# Data Analysis of City Bikes in the Bay Area

* http://www.bayareabikeshare.com
* https://www.fordgobike.com/system-data

<table class="tg">
  <tr>
    <th class="tg-yw4l"><img src="./images/prediction_bikeSupply_SanFranciscoCaltrain.png"></th>
    <th class="tg-yw4l"><img src="./images/prediction_bikeSupply_SanFranciscoCaltrain_2015-06-11.png"></th>
  </tr>
</table>

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import pytz
from pytz import timezone
import dill as pickle

import fetchData as fd
import exploratoryAnalysis as expAn
import distanceMatrix

In [2]:
import sys
sys.version

'3.6.3 | packaged by conda-forge | (default, Oct  5 2017, 14:07:33) \n[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]'

In [3]:
stationName = 'San Francisco Caltrain (Townsend at 4th)'  
#stationName = 'Market at Sansome' 

freq_hour = 1  ## sampling frequency to cumulate the nb. of bike trips
 
paths = [r'../data/babs_open_data_year_2']

## Get, Read and Clean raw data

In [4]:
## Uncomment to download and extract the data
!mkdir -p ../data
directory = "../data/"

links = ["https://s3.amazonaws.com/babs-open-data/babs_open_data_year_1.zip",
         "https://s3.amazonaws.com/babs-open-data/babs_open_data_year_2.zip",
         "https://s3.amazonaws.com/babs-open-data/babs_open_data_year_3.zip"]
for url in links:
    (filename, path_to_file) = fd.download_file_by_url(url, directory)
    fd.unzip_file(filename, path_to_file)

In [5]:
!ls ../data/babs_open_data_year_2

201508_station_data.csv  201508_trip_data.csv	  babs_open_data_year_2.zip
201508_status_data.csv	 201508_weather_data.csv  README.txt


In [6]:
# Making sure we get the time from the right timezone 
parse = lambda x: pytz.timezone('US/Pacific').localize(datetime.strptime(x,'%m/%d/%Y %H:%M')) #.astimezone(pytz.timezone('US/Pacific'))
print(parse("8/31/2015 23:26").astimezone(pytz.timezone('Europe/Paris')))

print("Now in Oslo (Norway): ", pytz.timezone('Europe/Oslo').localize(datetime.now()))
print("Now in San Francisco: ", pytz.timezone('Europe/Oslo').localize(datetime.now()).astimezone(pytz.timezone('America/Los_Angeles')))

2015-09-01 08:26:00+02:00
Now in Oslo (Norway):  2017-10-08 21:06:41.054647+02:00
Now in San Francisco:  2017-10-08 12:06:41.054983-07:00


## Trips data

In [7]:
## BikeTrips data (takes a few minutes!!)
dataTrips = expAn.read_or_store_object('dataTrips', './pkl', expAn.read_dataTrips_from_csv_files, paths=paths, asUTC=False)

In [8]:
## SUPPLY & DEMAND: Count how many bikes are arriving/leaving per x hour at a given station
cumulEndTrips   = expAn.read_or_store_object('cumulEndTrips', './pkl',
                                             expAn.count_bikes_arriving, dataTrips, stationName, freq_hour)
cumulStartTrips = expAn.read_or_store_object('cumulStartTrips', './pkl',
                                             expAn.count_bikes_leaving, dataTrips, stationName, freq_hour)

In [9]:
dataTrips.dtypes

Trip ID                     int64
Duration                    int64
Start Date         datetime64[ns]
Start Station              object
Start Terminal              int64
End Date           datetime64[ns]
End Station                object
End Terminal                int64
Bike #                      int64
Subscriber Type            object
Zip Code                   object
dtype: object

In [10]:
dataTrips.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
0,913460,765,2015-08-31 23:26:00,Harry Bridges Plaza (Ferry Building),50,2015-08-31 23:39:00,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
1,913459,1036,2015-08-31 23:11:00,San Antonio Shopping Center,31,2015-08-31 23:28:00,Mountain View City Hall,27,35,Subscriber,95032
2,913455,307,2015-08-31 23:13:00,Post at Kearny,47,2015-08-31 23:18:00,2nd at South Park,64,468,Subscriber,94107
3,913454,409,2015-08-31 23:10:00,San Jose City Hall,10,2015-08-31 23:17:00,San Salvador at 1st,8,68,Subscriber,95113
4,913453,789,2015-08-31 23:09:00,Embarcadero at Folsom,51,2015-08-31 23:22:00,Embarcadero at Sansome,60,487,Customer,9069


## Stations data

In [11]:
stationData  = expAn.read_dataStation_from_csv_files(paths)
stationID    = expAn.get_stationID(stationName, stationData)
stationDockcount = expAn.get_stationDockcount(stationName, stationData)
(lat, lon)    = expAn.get_station_coordinates(stationName, stationData)
neighboursIDs = expAn.get_neighbouring_stationIDs(stationData, stationName, radius=.65)
print("IDs of neighbouring stations to {statName}: {IDs}".format(statName=stationName, IDs=neighboursIDs))
print("Names stations nearby {statName}:\n{Names}".format(statName=stationName,
                                                                   Names=expAn.get_stationNames(neighboursIDs, stationData)))

IDs of neighbouring stations to San Francisco Caltrain (Townsend at 4th): [61 69]
Names stations nearby San Francisco Caltrain (Townsend at 4th):
['2nd at Townsend', 'San Francisco Caltrain 2 (330 Townsend)']


## Weather data

In [12]:
dateparse = lambda x: pytz.timezone('US/Pacific').localize(datetime.strptime(x,'%m/%d/%Y')).astimezone(timezone('UTC'))

In [13]:
dateparse('9/1/2014')

datetime.datetime(2014, 9, 1, 7, 0, tzinfo=<UTC>)

In [14]:
df = pd.read_csv('../data/babs_open_data_year_2/201508_weather_data.csv', index_col=0, date_parser=dateparse)

In [28]:
weatherData = expAn.read_dataWeather_from_csv_files(paths=paths)
print("postal codes available with weather forecast: ", weatherData['Zip'].unique())
(precipitation_mm, temperature_celcius) = expAn.get_weatherInfos(weatherData, stationData, stationName)
weatherInfos = {}
weatherInfos['precipitation_mm']    = precipitation_mm
weatherInfos['temperature_celcius'] = temperature_celcius

postal codes available with weather forecast:  [94107 94063 94301 94041 95113]
post codes of neighborhood:  [94158, 94103, 94105, 94107]


## Predictions for a given station:

* Build a prediction model for the **supply** and **demand** of bikes which takes into account:
    * earlier samples,
    * the weather data,
    * possibly information about neighbour stations


### Predict the number of bikes arriving, and compare various algorithms

* linear regression
* decision tree regressor
* Random Forest
* Boosted decision tree


In [29]:
inputsDemand = [{'title':'Prediction of bike demand from {stationName}'.format(stationName=stationName),
                 'data':cumulStartTrips,
                 'freq_hour':freq_hour, 'supply_demand':'demand', 'checkDockAvailable':False,
                 'withWeather':True,
                 'label':'Linear Regression', 'algo':'LinearRegression',
                 'ratioTest':0.3, }]

In [30]:
inputsSupply = [{
                 'title':'Prediction of bike supply @ {stationName}'.format(stationName=stationName),
                 'data' : cumulEndTrips, 
                 'freq_hour':freq_hour, 'supply_demand':'supply', 'checkDockAvailable':True,
                 'withWeather': True,
                 'label':'Linear Regression', 'algo':'LinearRegression',
                 'ratioTest':0.3,
                 }]

### Description of the features in this simple autoregressive model

In [31]:
offsetsTrip = [pd.DateOffset(**k) for k in [{'hours': 1*freq_hour},{'hours': 2*freq_hour},\
                                            {'hours': 3*freq_hour}, {'hours': 8*freq_hour},\
                                            {'days': 1}, {'days': 7}, {'days': 14},\
                                            {'days': 28}]]
# weatherInfos['precipitation_mm'].index.asof(str(date(2013,8,29)))
print(date(2013,8,29))
earliestTime =  date(2013,8,29) + offsetsTrip[-1]
print("earliestTime: ",earliestTime)
print([earliestTime-i for i in offsetsTrip])


2013-08-29
earliestTime:  2013-09-26 00:00:00
[Timestamp('2013-09-25 23:00:00'), Timestamp('2013-09-25 22:00:00'), Timestamp('2013-09-25 21:00:00'), Timestamp('2013-09-25 16:00:00'), Timestamp('2013-09-25 00:00:00'), Timestamp('2013-09-19 00:00:00'), Timestamp('2013-09-12 00:00:00'), Timestamp('2013-08-29 00:00:00')]


In [32]:
weatherInfos.keys()

dict_keys(['precipitation_mm', 'temperature_celcius'])

In [33]:
(X_full, y_full) = expAn.build_modelInputs(inputsSupply[0], weatherInfos)

In [34]:
def build_split_train_predict(inputs, weatherInfos={}):
    from sklearn import preprocessing
    from sklearn.linear_model import LinearRegression
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
    outputs = []
    for inp in inputs:
        ## READ AND COMPLETE INPUTS WITH DEFAULT VALUES IF MISSING
        inpDF = inp['data']
        output = inp
        if 'algo' not in inp.keys():
            output['algo'] = 'LinearRegression'
        if 'ratioTest' not in inp.keys():
            output['ratioTest'] = 0.3
        if 'withWeather' not in inp.keys():
            output['withWeather'] = True
        if 'checkDockAvailable' not in inp.keys():
            output['checkDockAvailable'] = True
        if 'checkBikeAvailable' not in inp.keys():
            output['checkBikeAvailable'] = True
        ## build the model
        (X_full, y_full) = expAn.build_modelInputs(inp, weatherInfos)

        # Split the data into training/testing sets
        nTest = int(round(len(X_full)*output['ratioTest']))
        X_trainingSet = X_full[:-nTest]
        X_testingSet  = X_full[-nTest:]
        # Split the targets into training/testing sets
        y_train = y_full[:-nTest]
        y_test  = y_full[-nTest:]
        output['X_full'] = X_full
        output['y_full'] = y_full
        output['X_trainingSet'] = X_trainingSet
        output['X_testingSet'] = X_testingSet
        output['y_train'] = y_train
        output['y_test'] = y_test
        output['dates_train'] = inpDF.index[:-nTest]
        output['dates_test'] = inpDF.index[-nTest:]
        print(inp['algo'])

        ## Normalizing the model coefficients for meaningful comparison
        ## Standardization, or mean removal and variance scaling
        ## (http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)
        scaler = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True).fit(X_trainingSet)
        X_trainingSet = scaler.transform(X_trainingSet)
        X_testingSet = scaler.transform(X_testingSet)
        
        # Train the model using the training sets
        print('\nLabel: {}'.format(output['label']))
        
        algo = {'LinearRegression':LinearRegression()}[inp['algo']]        
        model = algo.fit(X_trainingSet, y_train)
        output['model'] = model
        
        RSS = sum((y_train - model.predict(X_trainingSet))**2)  ## residual sum of squares (RSS) p.62
        TSS = sum((y_train - np.mean(y_train))**2)
        
        ## The residual standard error (RSE) provides an absolute measure of lack of fit of the model to the data.
        RSE = (RSS/(len(y_train)-2))**(0.5) ## residual standard error (RSE) p.69
        
        ## But since the RSE is measured in the units of Y , it is not always clear what constitutes a good RSE.
        ## The R**2 statistic provides an alternative measure of fit. It takes the form of a proportion
        ## — the proportion of variance explained — and so it always takes on a value between 0 and 1, and is 
        ## independent of the scale of Y. (p.69-70)
        R2statistic = 1. - RSS/TSS
        
        # prediction
        y_predict = model.predict(X_testingSet) # to compare to the real values: y_test
        output['y_predict'] = y_predict

        ### Explained variance score: 1 is perfect prediction
        #if inp['algo'] != 'RandomForestRegressor':
        scoreVarTrain = model.score(X_trainingSet, y_train)
        scoreVarTest = model.score(X_testingSet, y_test)
        sse = np.mean((y_predict - y_test) ** 2)
        print("sum of squared errors of prediction (SSE): %.2f" % sse)
        print('Variance score on training set: %.2f' % scoreVarTrain)
        print('Variance score on test set: %.2f' % scoreVarTest)
        print('RSE: %.2f' % RSE)
        print('R^2: %.2f' % R2statistic)
        stats = {'scoreVarTrain':scoreVarTrain,
                 'scoreVarTest':scoreVarTest, 'sse':sse,
                 'RSS':RSS, 'TSS':TSS, 'RSE':RSE,
                 'R2statistic':R2statistic}
        output['stats'] = stats
        outputs.append(output)
    return outputs


In [35]:
results_supply = build_split_train_predict(inputsSupply, weatherInfos)

LinearRegression

Label: Linear Regression
sum of squared errors of prediction (SSE): 7.64
Variance score on training set: 0.83
Variance score on test set: 0.83
RSE: 3.21
R^2: 0.83


In [36]:
results_demand = build_split_train_predict(inputsDemand, weatherInfos)

LinearRegression

Label: Linear Regression
sum of squared errors of prediction (SSE): 5.30
Variance score on training set: 0.80
Variance score on test set: 0.82
RSE: 2.75
R^2: 0.80


### Plot to compare the combinations of input parameters and AI algo.

In [37]:
from bokeh.io import push_notebook, show, output_notebook, output_file
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.ranges import Range1d
from bokeh.models import LinearAxis
from bokeh.palettes import brewer
output_notebook()

Loading BokehJS ...

In [38]:
def plot_prediction(outputs, title="", weatherInfos={}):
    timeUTC = outputs[0]['dates_test'].tolist()
    timePST = [pytz.timezone('UTC').localize(x).astimezone(timezone('US/Pacific')) for x in timeUTC]
    actual_data = outputs[0]['y_test']
    temper = [weatherInfos['temperature_celcius'][weatherInfos['temperature_celcius'].index.asof(i)] for i in outputs[0]['dates_test']]
    precip = [weatherInfos['precipitation_mm'][weatherInfos['precipitation_mm'].index.asof(i)] for i in outputs[0]['dates_test']]
    
    p = figure(title=title, #toolbar_location=None,
               plot_width=800, plot_height=600, x_axis_type='datetime')
    p.y_range = Range1d(0, 45.)
    p.extra_y_ranges = {"Precipitations": Range1d(start=0, end=5)}
    p.add_layout(LinearAxis(y_range_name="Precipitations"), 'right')
    p.grid.grid_line_color = None
#     p.background_fill_color = "#eeeeee"
    for output in outputs:
        p.line(timePST, np.array(output['y_predict']),
               line_width=2, line_color="green")
        p.circle(timePST, np.array(output['y_predict']), fill_color="green", line_color="green", size=6,
                 legend=output['label'])
    p.line(timePST, np.array(temper), line_width=2, line_color="red", legend='Temp. (C)')
    p.line(timePST, precip, y_range_name='Precipitations', line_width=2, line_color="blue",  legend='Rain (mm)')
    
    p.patches(timePST, precip, color="blue", alpha=0.5, line_color='blue')
    
    
    
    p.line(timePST, actual_data, line_width=2, line_color="gray", line_dash="4 4")
    p.circle(timePST, actual_data, fill_color="gray", line_color="gray", size=6, legend='Actual', line_dash="4 4")
    p.xaxis.axis_label = 'Time'
    p.xaxis.formatter = DatetimeTickFormatter(days=["%a\n%d %b"])
    p.yaxis.axis_label = 'Bikes'
    output_file("predictions.html", title="Bay Area citybikes")
    show(p)

In [39]:
title = 'cityBikes/Prediction of bike supply @ {stationName}'.format(stationName=stationName)
plot_prediction(results_supply,title = title, weatherInfos=weatherInfos)

W-1005 (SNAPPED_TOOLBAR_ANNOTATIONS): Snapped toolbars and annotations on the same side MAY overlap visually: Figure(id='a52983a3-8adb-4e72-8e1e-80eeb83d4730', ...)


W-1005 (SNAPPED_TOOLBAR_ANNOTATIONS): Snapped toolbars and annotations on the same side MAY overlap visually: Figure(id='a52983a3-8adb-4e72-8e1e-80eeb83d4730', ...)
